[View in Colaboratory](https://colab.research.google.com/github/dkanunnikov/netology/blob/master/dk_cv4.ipynb)

# Yelp Restaurant Photo Classification

Описание задания:
Предсказание тегов ресторанов по фотографиям
Обучить модель предсказания тегов ресторанов по набору фото. Исходные данные и валидация на Kaggle в рамках контеста Yelp Restaurant Photo Classification. Шаблон ноутбука для подготовки решения: yelp-hw.ipynb .
Решение необходимо прислать в виде ссылки на ipython-ноутбука с указанием значения метрики на Leaderboard. Задание засчитывается при значение метрики на Leaderboard больше 0.7.

https://www.kaggle.com/c/yelp-restaurant-photo-classification

In [41]:
!pip install keras --upgrade
!pip install keras_applications --upgrade
!pip install kaggle


Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.2.2)
Requirement already up-to-date: keras_applications in /usr/local/lib/python3.6/dist-packages (1.0.4)


In [0]:
import os
import cv2
import numpy as np
import pandas as pd
#from tensorflow.contrib import keras
import keras
from keras.models import Model, Sequential
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten, Concatenate
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras import backend as K


preprocess_input = keras.applications.vgg16.preprocess_input
VGG16 = keras.applications.VGG16

IMG_SIZE = (224, 224)

In [0]:
!pip install kaggle-cli > /dev/null
!rm -rf train_photos && rm -rf test_photos > /dev/null && rm -rf train > /dev/null && rm -rf test > /dev/null


In [44]:
!kaggle  config  set  -n  path  -v  /content

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 7, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.6/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 109, in authenticate
    self._load_config(config_data)
  File "/usr/local/lib/python3.6/dist-packages/kaggle/api/kaggle_api_extended.py", line 151, in _load_config
    raise ValueError('Error: Missing %s in configuration.' % item)
ValueError: Error: Missing username in configuration.


In [45]:
!kg  download -u webotdel@mail.ru  -p 64136413L   -c yelp-restaurant-photo-classification -f train_photos.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train_photos.tgz

train_photos.tgz already downloaded !


In [46]:
!kg  download -u *** -p ***  -c yelp-restaurant-photo-classification -f test_photos.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/test_photos.tgz

test_photos.tgz already downloaded !


In [47]:
!kg  download -u *** -p ***  -c yelp-restaurant-photo-classification -f train.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train.csv.tgz

train.csv.tgz already downloaded !


In [48]:
!kg  download -u *** -p ***  -c yelp-restaurant-photo-classification -f train_photo_to_biz_ids.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/train_photo_to_biz_ids.csv.tgz

train_photo_to_biz_ids.csv.tgz already downloaded !


In [49]:
!kg  download -u *** -p ***  -c yelp-restaurant-photo-classification -f test_photo_to_biz.csv.tgz

/usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py:37: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 37 of the file /usr/local/lib/python3.6/dist-packages/mechanicalsoup/browser.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  response.content, **soup_config)
downloading https://www.kaggle.com/c/yelp-restaurant-photo-classification/download/test_photo_to_biz.csv.tgz

test_photo_to_biz.csv.tgz already downloaded !


In [0]:
!tar -zxvf train_photos.tgz > /dev/null && tar -zxvf test_photos.tgz > /dev/null

In [0]:
!tar -zxvf train.csv.tgz > /dev/null

In [0]:
!tar -zxvf train_photo_to_biz_ids.csv.tgz > /dev/null

In [0]:
!tar -zxvf test_photo_to_biz.csv.tgz > /dev/null

In [54]:
!ls  -l

total 13963456
drwxr-xr-x 3 root root        4096 Aug  3 16:51 datalab
-rw-r--r-- 1 root root      174767 Aug  8 11:12 submission.csv
drwxr-xr-x 2  501 staff   14315520 Oct 30  2015 test_photos
-rw-r--r-- 1 root root  7102584231 Aug  8 08:33 test_photos.tgz
-rw-r--r-- 1  504 staff   15193591 Dec 17  2015 test_photo_to_biz.csv
-rw-r--r-- 1 root root     5019061 Aug  8 08:33 test_photo_to_biz.csv.tgz
-rw-r--r-- 1  501 staff      28760 Dec 15  2015 train.csv
-rw-r--r-- 1 root root        7287 Aug  8 08:33 train.csv.tgz
drwxr-xr-x 2  501 staff   14131200 Oct 30  2015 train_photos
-rw-r--r-- 1 root root  7026724672 Aug  8 08:29 train_photos.tgz
-rw-r----- 1  501 staff    2698404 Dec 14  2015 train_photo_to_biz_ids.csv
-rw-r--r-- 1 root root     1171455 Aug  8 08:33 train_photo_to_biz_ids.csv.tgz
-rw-r--r-- 1 root root   116496080 Aug  8 08:40 yelp_weights.h5


## Загружаем разметку

In [0]:
train_biz_df = pd.read_csv('train.csv')
train_photos_df = pd.read_csv('train_photo_to_biz_ids.csv')
data_df = train_photos_df.merge(train_biz_df)

data_df.dropna(inplace=True)

test_photos_df = pd.read_csv('test_photo_to_biz.csv')



In [0]:
def get_image(path, img_id, img_size=(224, 224)):
    img = cv2.imread(os.path.join(path, '%s.jpg' % img_id))[:,:,::-1]
    img = preprocess_input(img.astype(np.float32))
    return cv2.resize(img, img_size)

def to_dense(labels):
    result = [0] * 9
    for i in labels:
        result[i] = 1.
    return result

def train_generator(df, img_size=(224, 224), batch_size=32):
    while True:
        df = df.sample(frac=1).reset_index(drop=True)
        for i in range(0, len(df) // batch_size * batch_size, batch_size):
            X, y = [], []
            for _, row in df[i:i + batch_size].iterrows():
                X.append(get_image('train_photos', row['photo_id'], img_size))
                y.append(to_dense(map(int, str(row['labels']).split())))
            y = np.array(y)
            yield np.array(X), [y[:, i] for i in range(9)]

In [0]:
# TODO: build CNN model
# TODO: train CNN model using train_generator

In [0]:

from keras.callbacks import ModelCheckpoint

base_model = VGG16(
    include_top = False,
    weights = 'imagenet',
    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
    pooling = 'avg'
)

checkout = ModelCheckpoint('yelp_weights.h5', save_best_only=True, verbose=1)

In [0]:


#base_model = InceptionV3(
#    include_top = False,
#    weights = 'imagenet',
#    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
#    pooling = 'avg'
#)

#base_model = Xception(include_top = False,
#                   weights = 'imagenet',
#                   input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
#                   pooling = 'avg'
#           )


In [0]:
# фиксируем все веса предобученной сети
for layer in base_model.layers:
    layer.trainable = False

In [60]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
x = base_model.output
predictions = []
for i in range(9):
    x0 = Dense(128, activation='relu')(x)
    x0 = Dropout(0.5)(x0)
    x0 = Dense(1, activation='sigmoid')(x0)
    predictions.append(x0)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(data_df, test_size=0.01, random_state=123)
test_X, test_Y = next(train_generator(test_df, batch_size = len(test_df)))

In [0]:
!rm yelp_weights.h5

In [64]:
model.fit_generator(
        train_generator(train_df, batch_size = 100),
        steps_per_epoch=100,
        epochs=10,
        validation_data=(test_X, test_Y),
        callbacks=[checkout]
)

Epoch 1/10
100/100 [==============================] - 153s 2s/step - loss: 9.8834 - dense_20_loss: 1.5234 - dense_22_loss: 0.9963 - dense_24_loss: 1.0207 - dense_26_loss: 1.1845 - dense_28_loss: 0.9369 - dense_30_loss: 1.3030 - dense_32_loss: 0.8530 - dense_34_loss: 1.0120 - dense_36_loss: 1.0537 - dense_20_acc: 0.7131 - dense_22_acc: 0.6435 - dense_24_acc: 0.6622 - dense_26_acc: 0.5149 - dense_28_acc: 0.6386 - dense_30_acc: 0.6986 - dense_32_acc: 0.7497 - dense_34_acc: 0.6398 - dense_36_acc: 0.6233 - val_loss: 4.9500 - val_dense_20_loss: 0.4891 - val_dense_22_loss: 0.5624 - val_dense_24_loss: 0.5370 - val_dense_26_loss: 0.6939 - val_dense_28_loss: 0.5641 - val_dense_30_loss: 0.5080 - val_dense_32_loss: 0.4265 - val_dense_34_loss: 0.5761 - val_dense_36_loss: 0.5929 - val_dense_20_acc: 0.7762 - val_dense_22_acc: 0.7106 - val_dense_24_acc: 0.7276 - val_dense_26_acc: 0.5494 - val_dense_28_acc: 0.7080 - val_dense_30_acc: 0.7570 - val_dense_32_acc: 0.8150 - val_dense_34_acc: 0.7089 - val_de

In [65]:
!ls -l


total 13914288
drwxr-xr-x 3 root root        4096 Aug  3 16:51 datalab
-rw-r--r-- 1 root root      174767 Aug  8 11:12 submission.csv
drwxr-xr-x 2  501 staff   14315520 Oct 30  2015 test_photos
-rw-r--r-- 1 root root  7102584231 Aug  8 08:33 test_photos.tgz
-rw-r--r-- 1  504 staff   15193591 Dec 17  2015 test_photo_to_biz.csv
-rw-r--r-- 1 root root     5019061 Aug  8 08:33 test_photo_to_biz.csv.tgz
-rw-r--r-- 1  501 staff      28760 Dec 15  2015 train.csv
-rw-r--r-- 1 root root        7287 Aug  8 08:33 train.csv.tgz
drwxr-xr-x 2  501 staff   14131200 Oct 30  2015 train_photos
-rw-r--r-- 1 root root  7026724672 Aug  8 08:29 train_photos.tgz
-rw-r----- 1  501 staff    2698404 Dec 14  2015 train_photo_to_biz_ids.csv
-rw-r--r-- 1 root root     1171455 Aug  8 08:33 train_photo_to_biz_ids.csv.tgz
-rw-r--r-- 1 root root    66147176 Aug  8 11:56 yelp_weights.h5


## Получаем предсказания

In [0]:
model.load_weights('yelp_weights.h5')

In [67]:
len(test_photos_df)

1190225

In [0]:
class TestSequence(keras.utils.Sequence):
    """ Класс для чтения батча """
    def __init__(self, df, batch_size):
        self._df = df
        self._batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self._df) / float(self._batch_size)))

    def __getitem__(self, idx):
        result = []
        sample = self._df[idx * self._batch_size:(idx + 1) * self._batch_size]
        for _, row in sample.iterrows():
            result.append(get_image('test_photos', row['photo_id']))
        return np.array(result)

In [69]:
test_photos_df.head()

,photo_id,business_id
0,317818,003sg
1,30679,003sg
2,455084,003sg
3,371381,003sg
4,86224,003sg


In [70]:
# читаем данные параллельно в 4потока и применяем модель
preds = model.predict_generator(TestSequence(test_photos_df, batch_size=100), workers=4, verbose=1)

   

11903/11903 [==============================] - 12727s 1s/step


In [0]:
test_photos_df['labels'] = [[float(preds[j][i]) for j in range(9)] 
                            for i in range(len(test_photos_df))] 

In [72]:
test_photos_df.head()

,photo_id,business_id,labels
0,317818,003sg,"[0.20483800768852234, 0.467350572347641, 0.522..."
1,30679,003sg,"[0.3618176579475403, 0.49222323298454285, 0.40..."
2,455084,003sg,"[0.2874740958213806, 0.33303380012512207, 0.29..."
3,371381,003sg,"[0.1992282122373581, 0.5290882587432861, 0.573..."
4,86224,003sg,"[0.04263046011328697, 0.9716522097587585, 0.98..."


In [0]:
business_df = test_photos_df.groupby('business_id')

In [0]:
submission = []
for k, labels in business_df['labels'].apply(list).iteritems():
    labels = np.array(labels).mean(axis=0)
    labels = np.where(labels > 0.5)[0]
    submission.append((k, ' '.join(map(str, labels))))

In [0]:
submission_df = pd.DataFrame(submission, columns=('business_id', 'labels'))
submission_df.to_csv('submission.csv', index=False)
# 0.73373

In [76]:
!ls -l

total 13914276
drwxr-xr-x 3 root root        4096 Aug  3 16:51 datalab
-rw-r--r-- 1 root root      163329 Aug  8 15:28 submission.csv
drwxr-xr-x 2  501 staff   14315520 Oct 30  2015 test_photos
-rw-r--r-- 1 root root  7102584231 Aug  8 08:33 test_photos.tgz
-rw-r--r-- 1  504 staff   15193591 Dec 17  2015 test_photo_to_biz.csv
-rw-r--r-- 1 root root     5019061 Aug  8 08:33 test_photo_to_biz.csv.tgz
-rw-r--r-- 1  501 staff      28760 Dec 15  2015 train.csv
-rw-r--r-- 1 root root        7287 Aug  8 08:33 train.csv.tgz
drwxr-xr-x 2  501 staff   14131200 Oct 30  2015 train_photos
-rw-r--r-- 1 root root  7026724672 Aug  8 08:29 train_photos.tgz
-rw-r----- 1  501 staff    2698404 Dec 14  2015 train_photo_to_biz_ids.csv
-rw-r--r-- 1 root root     1171455 Aug  8 08:33 train_photo_to_biz_ids.csv.tgz
-rw-r--r-- 1 root root    66147176 Aug  8 11:56 yelp_weights.h5


In [0]:
from google.colab import files
files.download('submission.csv')